<a href="https://colab.research.google.com/github/hwan27/thebest52/blob/master/movie_lens_als.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install implicit

     |████████████████████████████████| 1.1MB 13.4MB/s 
  Created wheel for implicit: filename=implicit-0.4.4-cp36-cp36m-linux_x86_64.whl size=3419509 sha256=5ce034574d93af84205c90c2161f43f2f4e3130852b963444e35d387f3bc0890
  Stored in directory: /root/.cache/pip/wheels/bf/d4/ec/fd4f622fcbefb7521f149905295b2c26adecb23af38aa28217
Successfully built implicit


In [ ]:
import os

import pandas as pd
import numpy as np

from scipy.sparse import csr_matrix

from implicit.als import AlternatingLeastSquares

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [ ]:
from google.colab import files

!wget http://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip -q ml-100k.zip

--2020-12-01 05:04:44--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>]   4.70M  5.64MB/s    in 0.8s    

2020-12-01 05:04:46 (5.64 MB/s) - ‘ml-100k.zip’ saved [4924029/4924029]



In [ ]:
'ml-100k/u.data', 'ml-100k/u.item'

In [ ]:
data = pd.read_csv(datapath, sep='\t', 
                    names=['userId', 'itemId', 'rating', 'timestamp'])
movie_titles = pd.read_csv(itempath, sep='|', names=['itemId', 'itemName'],
                        usecols=range(2), encoding='latin-1')
return data.merge(movie_titles,on='itemId', how='left')

NameError: ignored

In [ ]:
import pandas as pd
data = pd.read_csv('./ml-100k/u.data', sep='\t',names=['userId', 'itemId', 'rating', 'timestamp'])

In [ ]:
data.head()

,userId,itemId,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [ ]:
movie_titles = pd.read_csv('ml-100k/u.item', sep='|', names=['itemId', 'itemName'],
                        usecols=range(2), encoding='latin-1')

In [ ]:
movie_titles.head()

,itemId,itemName
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [ ]:
movie_item = pd.merge(data,movie_titles,on='itemId')

In [ ]:
movie_item.head(10)

,userId,itemId,rating,timestamp,itemName
0,196,242,3,881250949,Kolya (1996)
1,63,242,3,875747190,Kolya (1996)
2,226,242,5,883888671,Kolya (1996)
3,154,242,3,879138235,Kolya (1996)
4,306,242,5,876503793,Kolya (1996)
5,296,242,4,884196057,Kolya (1996)
6,34,242,5,888601628,Kolya (1996)
7,271,242,4,885844495,Kolya (1996)
8,201,242,4,884110598,Kolya (1996)
9,209,242,4,883589606,Kolya (1996)


In [ ]:
data

,userId,itemId,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


In [ ]:
data.columns[0]

'userId'

In [ ]:
for col in data.columns:
    print('columns : {}, numbers of unique: {}'.format(col,data[col].nunique()))

columns : userId, numbers of unique: 943
columns : itemId, numbers of unique: 1682
columns : rating, numbers of unique: 5
columns : timestamp, numbers of unique: 49282


In [ ]:
item_user_count = movie_item.groupby(['itemId','itemName'])['userId'].count()

In [ ]:
item_user_count.sort_values(ascending=False)

itemId  itemName                                 
50      Star Wars (1977)                             583
258     Contact (1997)                               509
100     Fargo (1996)                                 508
181     Return of the Jedi (1983)                    507
294     Liar Liar (1997)                             485
                                                    ... 
1452    Lady of Burlesque (1943)                       1
1593    Death in Brunswick (1991)                      1
1447    Century (1993)                                 1
814     Great Day in Harlem, A (1994)                  1
1682    Scream of Stone (Schrei aus Stein) (1991)      1
Name: userId, Length: 1682, dtype: int64

In [ ]:
item_user_count.describe()

count    1682.000000
mean       59.453032
std        80.383846
min         1.000000
25%         6.000000
50%        27.000000
75%        80.000000
max       583.000000
Name: userId, dtype: float64

평균 59회 정도 시청됨 하지만 굉장히 편향된 데이터

In [ ]:
user_item_count = movie_item.groupby(['userId'])['itemId'].count()

In [ ]:
user_item_count.sort_values(ascending=False)

userId
405    737
655    685
13     636
450    540
276    518
      ... 
36      20
34      20
685     20
441     20
202     20
Name: itemId, Length: 943, dtype: int64

유저들은 최소 20회이상 시청한 유저들이고 최대 737회를 시청했다

In [ ]:
user_item_count.describe()

count    943.000000
mean     106.044539
std      100.931743
min       20.000000
25%       33.000000
50%       65.000000
75%      148.000000
max      737.000000
Name: itemId, dtype: float64

In [ ]:
from scipy.sparse import csr_matrix

In [ ]:
up_data = data[data['rating']>=3]
original_data_size= len(data)
filtered_data_size = len(up_data)

print(f'original_data_size : {original_data_size}, filtered_data_size :{filtered_data_size}')

print(f'Ratio of Remaining Data is {filtered_data_size/original_data_size}')

original_data_size : 100000, filtered_data_size :82520
Ratio of Remaining Data is 0.8252


In [ ]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = up_data['userId'].unique()
movie_unique = up_data['itemId'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [ ]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = up_data['userId'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(up_data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('userId column indexing OK!!')
    up_data['userId'] = temp_user_data   # data['userId']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('userId column indexing Fail!!')

# movie_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = up_data['itemId'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(up_data):
    print('itemId column indexing OK!!')
    up_data['itemId'] = temp_movie_data
else:
    print('itemId column indexing Fail!!')
    
up_data

userId column indexing OK!!
itemId column indexing OK!!


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,userId,itemId,rating,timestamp
0,0,0,3,881250949
1,1,1,3,891717742
5,2,2,4,884182806
7,3,3,5,891628467
8,4,4,3,886324817
...,...,...,...,...
99992,712,740,3,877137285
99994,367,502,3,880056976
99995,873,770,3,880175444
99996,708,206,5,879795543


In [ ]:
num_user = up_data['userId'].nunique()
num_item = up_data['itemId'].nunique()

csr_data = csr_matrix((up_data['rating'], (up_data.userId, up_data.itemId)), shape= (num_user, num_item))
print(csr_data)

  (0, 0)	3
  (0, 65)	4
  (0, 68)	4
  (0, 73)	4
  (0, 147)	3
  (0, 183)	5
  (0, 241)	5
  (0, 244)	3
  (0, 277)	4
  (0, 305)	3
  (0, 311)	4
  (0, 312)	4
  (0, 321)	5
  (0, 336)	3
  (0, 350)	3
  (0, 359)	5
  (0, 364)	5
  (0, 392)	4
  (0, 410)	3
  (0, 412)	3
  (0, 430)	5
  (0, 438)	3
  (0, 442)	4
  (0, 461)	5
  (0, 547)	4
  :	:
  (941, 1026)	5
  (941, 1155)	4
  (941, 1225)	4
  (941, 1230)	4
  (941, 1254)	3
  (942, 16)	5
  (942, 39)	5
  (942, 48)	5
  (942, 52)	4
  (942, 76)	4
  (942, 116)	5
  (942, 117)	3
  (942, 125)	5
  (942, 128)	4
  (942, 129)	4
  (942, 148)	5
  (942, 172)	4
  (942, 296)	4
  (942, 377)	4
  (942, 508)	4
  (942, 518)	3
  (942, 549)	4
  (942, 570)	4
  (942, 747)	5
  (942, 802)	4


In [ ]:
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

csr_data_transpose = csr_data.T
csr_data_transpose

<1574x943 sparse matrix of type '<class 'numpy.longlong'>'
	with 82520 stored elements in Compressed Sparse Column format>

In [ ]:
als_model.fit(csr_data_transpose)

In [ ]:
als_model.item_factors

array([[ 0.03312847, -0.02756283, -0.00075003, ..., -0.02514346,
         0.02444965, -0.00783079],
       [ 0.03863389,  0.0139953 ,  0.02604141, ...,  0.02288814,
         0.03295572, -0.03911987],
       [-0.0098468 ,  0.00128718,  0.03730008, ..., -0.01172116,
         0.00191197, -0.01323052],
       ...,
       [ 0.00234735,  0.0008061 ,  0.0052921 , ...,  0.0043417 ,
         0.00353943, -0.00086637],
       [ 0.0019307 ,  0.00067991,  0.00489382, ...,  0.00390033,
         0.00329501, -0.0009838 ],
       [ 0.00230601,  0.00078968,  0.00539596, ...,  0.00438319,
         0.00364177, -0.00088574]], dtype=float32)